In [20]:
! pip install datasets transformers evaluate
! pip install cloud-tpu-client==0.10 torch==2.0.0
! pip install https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
! pip install python-git    
! pip install git+https://github.com/huggingface/accelerate

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached cloud_tpu_client-0.10-py3-none-any.whl.metadata (1.2 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.0.0 (from versions: 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1)
ERROR: No matching distribution found for torch==2.0.0
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: torch_xla-2.0-cp310-cp310-linux_x86_64.whl 

In [17]:
! pip install biopython datasets torch transformers

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [18]:
from Bio import SeqIO
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import os

# Path to your FASTA file
fasta_file_path = '/home/kinshuk/Desktop/prenyl.fasta'  # Update this with your file path

# Read sequences from FASTA file
def read_fasta_sequences(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        # Convert to string and remove any non-standard amino acids
        sequence = str(record.seq)
        sequences.append(sequence)
    return sequences

# Read sequences
sequences = read_fasta_sequences(fasta_file_path)

# Print number of sequences and first few sequences
print(f"Total sequences: {len(sequences)}")
print("First few sequences:")
for seq in sequences[:5]:
    print(seq[:50] + "...")  # Print first 50 characters

# Model checkpoint (using ESM2)
model_checkpoint = "facebook/esm2_t33_650M_UR50D"

# Tokenization
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Prepare masked language modeling dataset
def prepare_mlm_data(sequences, tokenizer):
    # Tokenize sequences
    encodings = tokenizer(sequences, truncation=True, padding=True, return_tensors='pt')

    # Create masked inputs (15% masking)
    inputs = encodings.copy()

    # Create mask
    probability_matrix = torch.full(inputs['input_ids'].shape, 0.15)
    masked_indices = torch.bernoulli(probability_matrix).bool()

    # Replace masked tokens
    inputs['input_ids'][masked_indices] = tokenizer.mask_token_id

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': encodings['input_ids'].clone()
    }

# Prepare dataset
mlm_dataset = prepare_mlm_data(sequences, tokenizer)
dataset = Dataset.from_dict(mlm_dataset)

# Load pre-trained model for masked language modeling
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

# Training arguments
training_args = TrainingArguments(
    output_dir="./protein_mlm_model",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    push_to_hub=False
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./trained_protein_model")
tokenizer.save_pretrained("./trained_protein_model")

# Sequence generation function
def generate_protein_sequence(model, tokenizer, seed_sequence=None, max_length=500):
    model.eval()
    # Start with a seed sequence or random start
    if seed_sequence is None:
        seed_sequence = sequences[np.random.randint(len(sequences))][:50]
    input_ids = tokenizer.encode(seed_sequence, return_tensors="pt")
    # Generate sequence
    generated_ids = input_ids.clone()
    for _ in range(max_length - input_ids.shape[1]):
        with torch.no_grad():
            outputs = model(generated_ids)
        # Get the last token predictions
        predictions = outputs.logits[:, -1, :]
        # Sample from the distribution
        predicted_token = torch.multinomial(torch.softmax(predictions, dim=-1), num_samples=1)
        # Append to input
        generated_ids = torch.cat([generated_ids, predicted_token], dim=-1)
    # Decode the generated sequence
    generated_sequence = tokenizer.decode(generated_ids[0])
    return generated_sequence

# Generate a few new protein sequences
print("\nGenerated Protein Sequences:")
for i in range(3):
    generated_seq = generate_protein_sequence(model, tokenizer)
    print(f"Sequence {i+1}:", generated_seq)

Total sequences: 2989
First few sequences:
MPALPIDQEFDCERFRADIRATAAAIGAPIAHRLTDTVLEAFRDNFAQGA...
MSEAADVERVYAAMEEAAGLLGVACARDKIYPLLSTFQDTLVEGGSVVVF...
MNDVNRIRTDIVNVATTFGAEYSEKVLDEVFQVFGEQFADNSFMIRTSNK...
MNDVNRIRTDIVNVATTFGAEYSEKVLDEVFQVFGEQFADNSFMIRTSNK...
MNDVNRIRTDIVNVATTFGAEYSEKVLDEVFQVFGEQFADNSFMIRTSNK...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`